In [1]:
import sqlite3
import pandas as pd
from tqdm import tqdm
import random
from random import choice
import time
import os

In [71]:
class NewsDataCreater():
    """
    Create News training data
    """
    def __init__(self):
        self.name = 'NewsDataCreater'
        self.DB_PATH = '/home/u4839782/sentence-transformers/examples/datasets/News/NewsNetwork_ch.db'
        self.DB_SELECT_START_DATE = '2020-08-20'
        self.CSV_SAVE_PATH = './'
        
    def select_database(self, database_name, table_name, columns, conditions):
        conn = sqlite3.connect(database_name)
        cur = conn.cursor()
        myresult = cur.execute('''SELECT {1}
                                FROM {0} WHERE {2}'''
                               .format(table_name,columns,conditions) )
        result = []
        for row in myresult:
            result.append(row)
        conn.close()    
        return result

In [72]:
NDC = NewsDataCreater()

In [73]:
media_name_list=['CHT','BUSINESSWEEKLY','CNA','ETT','LBT','NOWNEWS','SETN','STORM','TVBS','UDN']

In [76]:
NBA_news = []
for media_name in media_name_list:
    NBA_news += NDC.select_database(NDC.DB_PATH, media_name+'news', '*', 
                                    "NewsTitle like '%NBA%' and NewsTitle like '%湖人%' and NewsDate >='2020-08-15' ")

In [77]:
len(NBA_news)

31

In [118]:
for i, k in enumerate(NBA_news):
    print(i, k[4])

0 NBA》林書豪版首輪預測 湖人4比2拓荒者 - 體育
1 NBA》湖人詹姆斯：今年季後賽苦戰必至 - 體育
2 阿拓乘勝追擊 湖人恐翻船 - NBA
3 NBA》拓荒者擒殺湖人 上演西區老八傳奇 - 體育
4 武漢肺炎蔓延NBA 湖人證實2球員感染 塞爾蒂克史瑪特確診
5 NBA傳球星杯葛復賽聲援示威 湖人不跟風
6 NBA》拓荒者能扳倒湖人？「愛神」洛夫點出關鍵人物
7 NBA》湖人拓荒者首輪有亮點！ 03梯兩大球星生涯二度交鋒
8 NBA》季後賽首輪強碰西霸天湖人 里拉德：我們有機會一搏
9 NBA》湖人小心了！球探：拓荒者是偽裝的第8種子
10 NBA》紀念Kobe 湖人首輪若晉級將換上黑曼巴球衣
11 NBA》湖人大利空！ 拓荒者殺手恐無法提前復出
12 NBA》湖人首輪被看衰恐爆冷出局 一眉哥霸氣回應
13 NBA》歷史數據看好湖人 上次系列戰輸拓荒者是28年前
14 NBA》復賽後命中率狂降 湖人「聖堂射手」找回好手感有信心
15 NBA季後賽1》湖人一面倒好評 最怕外線低迷
16 NBA》湖人利多！拓荒者箝制AD的內線戰力不會上場
17 NBA》湖人季後賽首輪強碰拓荒者 詹皇坦言今年很艱難
18 NBA》大爆冷！東西霸天公鹿、湖人季後賽首戰竟同日落難
19 NBA》詹皇大三元做白工 超強老八拓荒者扳倒湖人
20 NBA》若湖人一輪遊 「詹黑」皮爾斯：詹姆斯無法列入史上最佳
21 NBA》落後湖人傳喜訊！「拓荒者殺手」第二戰有望回歸
22 NBA》逆襲的老8！拓荒者明戰湖人拚連勝 里拉德：我們會更好
23 里拉德攻下34分 NBA拓荒者擊退湖人│TVBS新聞網
24 NBA／暴龍戰績超越湖人 本季勝率創隊史新高
25 NBA／拓荒者闖季後賽首戰湖人 甜瓜二度對決詹姆斯
26 NBA／里拉德狂飆34分 拓荒者末節三分雨轟垮湖人
27 NBA／湖人最後關頭失守 詹姆斯坦言攻守兩端出狀況
28 NBA／湖人三分外線陷低潮 庫茲瑪：得把記憶消除
29 NBA／紀念緬懷布萊恩 湖人曼巴日將穿特殊球衣出賽
30 NBA／湖人戴特殊紅色鴨舌帽 詹皇再喊話伸張正義


In [37]:
typhoon_news = []
for media_name in media_name_list:
    typhoon_news += NDC.select_database(NDC.DB_PATH, media_name+'news', '*', 
                                        "NewsTitle like '%巴威%' and NewsDate >='2020-08-20' ")

In [55]:
len(typhoon_news)

5

In [130]:
    def create_single_test_data(TWO_news):
        def news_paragraph_segmentation(context):
            news_length = len(context)
            paragraph_list = context.split('\n')
            paragraph_list = [k for k in paragraph_list if k!='']

            ### 比較兩種分段方式，哪種分得比較平均
            p_first_2=''
            p_second_2=''
            for s in paragraph_list:
                if len(p_first_2)<=int(news_length/2): p_first_2+=s
                else: p_second_2+=s

            p_first_1=''
            p_second_1=''
            for i in range(len(paragraph_list)):
                s = paragraph_list[len(paragraph_list)-i-1]
                if len(p_second_1)<=int(news_length/2): p_second_1=s+p_second_1
                else: p_first_1=s+p_first_1

            if len(p_first_2)==0 or len(p_second_2)==0:
                return p_first_1, p_second_1
            elif len(p_first_1)==0 or len(p_second_1)==0:
                return p_first_2, p_second_2
            else:
                if abs(len(p_first_2)/len(p_second_2)-1) < abs(len(p_first_1)/len(p_second_1)-1):
                    return p_first_2, p_second_2
                else:
                    return p_first_1, p_second_1
        
#         ALL_news = self.get_all_news()
        ALL_news_context = [k[-1] for k in TWO_news]
    
        test_data = [] # label: 0 --->同一篇新聞 切成2段
        for context in (ALL_news_context[:]):
            p1, p2 = news_paragraph_segmentation(context)
            p1 = p1.replace(',','')
            p2 = p2.replace(',','')
            if p1!='' and p2!='':
                test_data.append([p1, p2, 0])
        
        test_data.append([test_data[0][0], test_data[1][1], 1])
        test_data.append([test_data[0][1], test_data[1][0], 1])

        return test_data

In [134]:
test_data = []
test_data += create_single_test_data(NBA_news[1:3])
test_data += create_single_test_data(typhoon_news[0:2])
test_data += create_single_test_data([NBA_news[1], typhoon_news[0]])[2:]
test_data += create_single_test_data([NBA_news[2], typhoon_news[1]])[2:]

test_data += create_single_test_data([NBA_news[23], NBA_news[26]])
test_data += create_single_test_data(typhoon_news[2:4])
test_data += create_single_test_data([NBA_news[23], typhoon_news[2]])[2:]
test_data += create_single_test_data([NBA_news[26], typhoon_news[3]])[2:]

In [135]:
test_data

[['西區老八拓荒者「雙槍」英勇非常，讓老大湖人也感受到壓力，主將詹姆斯(LeBron James)認為這將會是他17年NBA生涯最艱苦的一次季後賽。「這很可能是非常困難的季後賽，甚至是最困難的。對我來說是最困難的一次總冠軍挑戰。」身經百戰的詹姆斯告訴《洛杉磯時報》說：「光是能打到現在的位置，我就覺得很可貴。我珍視善用每一天，這一切都不是理所當然的，特別這是我第17個球季了，我不會一開季就說：好，來打季後賽，來打總冠軍賽，我珍惜這所有的過程。」',
  '拓荒者從對戰成績來看絕不是湖人的對手，但是後場「雙槍」利拉德(Damian Lillard)、麥科倫(CJ McCollum)近況火熱。拓荒者本季的進攻評比高居全聯盟第3，雖然防守評比也是倒數第3，但他們就是擅長與對手互相飆分，也就是「換拳」的打法。外界認為這對湖人可能較為不利(雖然還是看好湖人晉級)，因為湖人是守優於攻，進攻能力不足(進攻評比第11)，特別是外線人才短缺，三分球命中率34.9%、場均出手數31.6都是聯盟後段班(排名第22與第23)，拓荒者全力防堵詹姆斯就有機會阻斷他們的火力供輸。(中時新聞網)',
  0],
 ['在奧蘭多隔離園區進行的NBA季後賽首輪賽事從今天起展開，在競爭激烈的西區，湖人與拓荒者的首輪系列戰最受關注，拓荒者甚至還被看好有機會上演「老八傳奇」，另外雷霆遭遇火箭，就看保羅能否上演成功復仇戲碼。湖人（1）對拓荒者（8）外界相當看好拓荒者上演老八傳奇，也就是第8種子淘汰第1種子戲碼，原因是兩隊在復賽階段狀態完全不同，湖人明顯還沒調整好，也仍在適應威特斯、J.R.史密斯兩名新成員，反觀拓荒者則是狀態火熱，特別是主控利拉德。就算湖人曾在季後賽系列戰碰過拓荒者11次，取得9勝2敗絕對優勢，至於本季兩隊3次對決，湖人也以2勝1敗占優，只是利拉德在復賽期間太過神勇，湖人恐怕很難找到合適防守人選，只能期望利拉德手感別再那麼火燙。',
  '快艇（2）對獨行俠（7）快艇本季例行賽碰上獨行俠3戰全勝，擺明吃定「歐陸雙雄」東契奇、波爾金吉斯領軍的獨行俠，加上快艇板凳暴徒哈瑞爾及時趕回奧蘭多隔離園區，可望能在首輪上場，讓快艇逐漸找回完整爭冠陣容，里歐納德表現也受矚目。金塊（3）對爵士（6）金塊本季面對爵士同樣取得3勝0敗對決優勢，除了後場主力陸續歸隊，小將麥克波特更在復賽階段打出亮眼表現，讓金塊可用之

In [136]:
pd.DataFrame(test_data).to_csv('/home/u4839782/bert-multi-gpu/data/news_paragraph_classification_2/0824_paragraph/test.csv', 
                               encoding='utf_8_sig', index=None, header=['paragraph', 'syn_paragraph', 'label'])